In [1]:
import syft as sy
import torch as th
import copy
import random

In [2]:
hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob")

In [3]:
@sy.func2plan(th.tensor([-1,-2]))
def plan_a(data):
    return data.abs()

@sy.func2plan(th.tensor([-1,-2]))
def plan_b(data):
    return -data.abs()

In [4]:
plan_a_ptr = plan_a.send(bob)
plan_b_ptr = plan_b.send(bob)

In [5]:
x = th.tensor([-1,2,3]).send(bob)

In [6]:
plan_a_res = plan_a_ptr(x)

[2, [1, [2, [[2, ['abs', [11, [69349253472, 77985372853, 'bob', None, [2]]], [2, []]]], 9745373178]]]]


In [7]:
plan_b_res = plan_b_ptr(x)

[2, [1, [2, [[2, ['abs', [11, [34656823993, 77985372853, 'bob', None, [2]]], [2, []]]], 89106720180]]]]
[2, [1, [2, [[2, ['__neg__', [11, [99414457078, 89106720180, 'bob', None, None]], [2, []]]], 9327024595]]]]
[2, [4, 89106720180]]


In [8]:
plan_a_res.get()

tensor([1, 2, 3])

In [9]:
plan_b_res.get()

tensor([-1, -2, -3])

In [10]:
bob._objects

{4304202390: <Plan id:4304202390 #tensors:0>,
 8330432230: <Plan id:8330432230 #tensors:0>,
 77985372853: tensor([-1,  2,  3])}